## Retriever And Chain With Langchain

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("ekaivakriti_pitch_deck.pdf")
pdf_docs = loader.load()


## Splitting

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
chunks_documents = text_splitter.split_documents(pdf_docs)

## Vector Embeddings & Vector Store using Faiss


In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(chunks_documents, OpenAIEmbeddings())
db

## Quering


In [ ]:
query = "contact details of ekaivakriti"
retrieved_results = db.similarity_search(query, k=1)
print(retrieved_results[0].page_content)

    

# Retriever & Chain
## Chat prompt Template


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant. Think step by step before answering, I will reward you if you provide me the correct and most relevant answer. Use the following pieces of context to answer the question at the end.
    <context>
    {context}
    </context>
    Question: {input}
"""
)


# Opensourse LLM


In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2", temperature=0.1)
llm


## Chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)
document_chain

In [ ]:
"""
Retreveral chain:   This chain retrieves relevant documents from the vector store based on the query and then uses the LLM to generate a response based on those documents.     
"""

from langchain.chains import create_retrieval_chain
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(
    retriever, document_chain
)

In [ ]:
response = retrieval_chain.invoke({"input": query})

print("\n🔍 Query:", response['input'])
print("\n📬 Answer:\n", response['answer'])

print("\n📄 Context Documents:\n")
for i, doc in enumerate(response['context'], 1):
    meta = doc.metadata
    print(f"--- Document {i} ---")
    print(f"Source: {meta.get('source')}")
    print(f"Page: {meta.get('page_label', meta.get('page'))}")
    content_preview = doc.page_content.strip().split('\n')[:5]  # Preview first 5 lines
    for line in content_preview:
        print(" ", line)
    print()
#response['answer']